'''<br>
    @Author: v sanjay kumar<br>
    @Date: 2024-09-06 04:00:30<br>
    @Last Modified by: v sanjay kumar<br>
    @Last Modified time: 2024-09-07 04:00:30.<br>
    @Title :All covid problems in pyspark <br>

'''


In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, round
from pyspark.sql.functions import coalesce, col


1.find death percentage  globaly

In [57]:
try:
    spark = SparkSession.builder \
        .appName("Find the global") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\covid_19_clean_complete.csv"
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    df.createOrReplaceTempView("country")
    result = spark.sql('''
    SELECT `Country/Region`, 
           ROUND((SUM(Deaths) * 100.0 / SUM(Confirmed)), 4) AS Death_percentage
    FROM country
    GROUP BY `Country/Region`
''')
    result.show()

except Exception as e:
    print(e)

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+----------------+
|Country/Region|Death_percentage|
+--------------+----------------+
|          Chad|          8.5993|
|      Paraguay|          1.0635|
|        Russia|          1.3640|
|         Yemen|         26.3575|
|       Senegal|          1.5353|
|    Cabo Verde|          1.0322|
|        Sweden|          9.0267|
|        Guyana|          7.0512|
|       Eritrea|          0.0000|
|   Philippines|          3.7305|
|         Burma|          2.5369|
|      Djibouti|          0.8956|
|      Malaysia|          1.4792|
|     Singapore|          0.0697|
|          Fiji

2.find death percentage  localy


In [5]:
try:
    spark = SparkSession.builder \
        .appName("Find the global") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\Covid Problem\covid_19_india.csv"
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.createOrReplaceTempView("covid_19_india")
    result=spark.sql('''
    SELECT `State/UnionTerritory`, 
           ROUND((SUM(CAST(Deaths AS INT)) * 100.0) / SUM(CAST(Confirmed AS INT)), 2) AS Death_percentage
    FROM covid_19_india
    GROUP BY `State/UnionTerritory`
    ORDER BY `State/UnionTerritory`
''')
    result.show()
    spark.stop()
    df.printSchema()
    
    


except Exception as e:
    print(e)

+--------------------+----------------+
|State/UnionTerritory|Death_percentage|
+--------------------+----------------+
|Andaman and Nicob...|            1.40|
|      Andhra Pradesh|            0.75|
|   Arunachal Pradesh|            0.37|
|               Assam|            0.64|
|               Bihar|            0.83|
|           Bihar****|            1.32|
|Cases being reass...|            0.00|
|          Chandigarh|            1.36|
|        Chhattisgarh|            1.26|
|Dadra and Nagar H...|            0.04|
|Dadra and Nagar H...|            0.05|
|         Daman & Diu|            0.00|
|               Delhi|            1.72|
|                 Goa|            1.59|
|             Gujarat|            1.55|
|             Haryana|            1.12|
|    Himachal Pradesh|            1.64|
|   Himanchal Pradesh|            1.71|
|   Jammu and Kashmir|            1.44|
|           Jharkhand|            1.21|
+--------------------+----------------+
only showing top 20 rows

root
 |-- Sno:

3.The countries with the highest infection rates


In [28]:
try:
    spark = SparkSession.builder \
        .appName("Find the global and local death percentage") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\country_wise_latest.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    # df.show()
    df.createOrReplaceTempView("country")
    result = spark.sql('''
    SELECT `Country/Region`, 
           ROUND((CAST(`Confirmed` AS FLOAT) / (SELECT SUM(CAST(`Confirmed` AS FLOAT)) FROM country)) * 100, 2) AS Infection_Rate_Percentage
    FROM country
    GROUP BY `Country/Region`, `Confirmed`
    ORDER BY Infection_Rate_Percentage DESC
''')

    result.show(10)
    
    spark.stop()

except Exception as e:
    print(e)


root
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: string (nullable = true)
 |-- Confirmed last week: integer (nullable = true)
 |-- 1 week change: integer (nullable = true)
 |-- 1 week % increase: double (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------------------+
|Country/Region|Infection_Rate_Percentage|
+--------------+-------------------------+
|            US|                    26.03|
|        Brazil|                    14.82|
|         India|                     8.98|
|        Russia|                     4.96|
|  South

4.The countries and continents with the highest death counts


In [30]:
try:
    spark = SparkSession.builder \
        .appName("Find hight death counts") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\country_wise_latest.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    # df.show()
    df.createOrReplaceTempView("country")
    result = spark.sql('''
        
    SELECT `Country/Region`,`WHO Region`, Deaths
    FROM country
    ORDER BY Deaths DESC;
    
''')

    result.show(10)
    
    spark.stop()

except Exception as e:
    print(e)


root
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: string (nullable = true)
 |-- Confirmed last week: integer (nullable = true)
 |-- 1 week change: integer (nullable = true)
 |-- 1 week % increase: double (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+--------------------+------+
|Country/Region|          WHO Region|Deaths|
+--------------+--------------------+------+
|            US|            Americas|148011|
|        Brazil|            Americas| 87618|
|United Kingdom|              Europe| 45844|
|        Mexico|            Americas| 4

5.Average number of deaths by day (Continents and Countries) 

In [6]:
try:
    spark = SparkSession.builder \
        .appName("Avarage number of deaths") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\worldometer_data.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    df.createOrReplaceTempView("worldometer_data")
    result = spark.sql('''
    SELECT
        `Country/Region`,
        Population,
        TotalCases,
        CASE 
            WHEN Population = 0 THEN NULL
            ELSE CAST(TotalCases AS FLOAT) / CAST(Population AS FLOAT)
        END AS CasesPerPopulation
    FROM
        worldometer_data
    WHERE
        Population RLIKE '^[0-9]+$'
        AND TotalCases RLIKE '^[0-9]+$'
        AND CAST(Population AS FLOAT) <> 0
    ORDER BY
        CasesPerPopulation DESC ''')
    result.show(10)

    spark.stop()


except Exception as e:
    print(e)

root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDe

 6.Average of cases divided by the number of population of each country (TOP 10)

In [12]:
try:
    spark = SparkSession.builder \
        .appName("Avarage of cases divided by the population") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\worldometer_data.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    df.createOrReplaceTempView("worldometer_data")
    result=spark.sql('''
    SELECT `Country/Region`,
           CAST(Population AS FLOAT) AS Population,
           CAST(TotalCases AS FLOAT) AS TotalCases,
           CASE 
               WHEN CAST(Population AS FLOAT) = 0 THEN NULL
               ELSE CAST(TotalCases AS FLOAT) / CAST(Population AS FLOAT)
           END AS CasesPerPopulation
    FROM worldometer_data
    WHERE Population RLIKE '^[0-9]+$'
      AND TotalCases RLIKE '^[0-9]+$'
      AND CAST(Population AS FLOAT) <> 0
    ORDER BY CasesPerPopulation DESC
''')
    result.show(10)

    spark.stop()


except Exception as e:
    print(e)


root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+------------+----------+--------------------+
|Country/Region|  Population|TotalCases|  CasesPerPopulation|
+--------------+------------+----------+--------------------+
|         Qatar|   2807805.0|  112092.0|0.039921575750452756|
| French Guiana|    299385.0|    8127

7. Considering the highest value of total cases, which countries have the highest rate of infection in relation to population?


In [14]:
try:
    spark = SparkSession.builder \
        .appName("Avarage of cases divided by the population") \
        .getOrCreate()

    # Corrected file path
    file_path = r"file:///C:\Users\sanju\OneDrive\Documents\dataset\worldometer_data.csv"
    

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    df.createOrReplaceTempView("worldometer_data")
    output=spark.sql('''
    SELECT `Country/Region`,
           CAST(Population AS FLOAT) AS Population,
           CAST(TotalCases AS FLOAT) AS TotalCases,
           CASE 
               WHEN CAST(Population AS FLOAT) = 0 THEN NULL
               ELSE (CAST(TotalCases AS FLOAT) * 10000) / CAST(Population AS FLOAT)
           END AS infliation_rate
    FROM worldometer_data
    WHERE Population RLIKE '^[0-9]+$'
      AND TotalCases RLIKE '^[0-9]+$'
      AND CAST(Population AS FLOAT) <> 0
    ORDER BY infliation_rate DESC
''')
    output.show()
    



except Exception as e:
    print(e)

root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+------------+----------+------------------+
|Country/Region|  Population|TotalCases|   infliation_rate|
+--------------+------------+----------+------------------+
|         Qatar|   2807805.0|  112092.0| 399.2157802981332|
| French Guiana|    299385.0|    8127.0| 271.

8.To find out the population vs the number of people vaccinated


In [15]:
try:
    spark = SparkSession.builder.appName("VaccinePercentage").getOrCreate()

    # Load data
    worldometer_data = spark.read.csv(r"file:///C:\Users\sanju\OneDrive\Documents\dataset\worldometer_data.csv", header=True, inferSchema=True)
    covid_vaccine_statewisenew = spark.read.csv(r"file:///C:\Users\sanju\OneDrive\Documents\dataset\Covid Problem\covid_vaccine_statewise.csv", header=True, inferSchema=True)
    # Get MaxPopulation for Asia
    max_population = worldometer_data.filter(worldometer_data.Continent == 'Asia').agg(max("Population").alias("MaxPopulation"))

    # Get the latest date for Total Doses Administered for India
    latest_date = covid_vaccine_statewisenew.filter(
        (covid_vaccine_statewisenew.State == 'India') &
        (covid_vaccine_statewisenew["Total Doses Administered"].isNotNull())
    ).agg(
        max("Updated On").alias("LatestDate")
    ).collect()[0]["LatestDate"]

    # Get Total Doses Administered for India on the latest date
    latest_doses = covid_vaccine_statewisenew.filter(
        (covid_vaccine_statewisenew.State == 'India') &
        (covid_vaccine_statewisenew["Updated On"] == latest_date)
    ).select("Total Doses Administered")

    # Calculate vaccine percentage
    vaccine_percentage = latest_doses.crossJoin(max_population).select(
        round((latest_doses["Total Doses Administered"] * 100.0) / max_population.MaxPopulation, 2).alias("vaccine_percentage")
    )

    vaccine_percentage.show()
    spark.stop()


except Exception as e:
    print(e)


+------------------+
|vaccine_percentage|
+------------------+
|             33.53|
+------------------+



9. To find out the percentage of different vaccine taken by people in a country


In [23]:
spark = SparkSession.builder.appName("VaccinePercentage").getOrCreate()

try:
    covid_vaccine_statewisenew = spark.read.csv(r"file:///C:\Users\sanju\OneDrive\Documents\dataset\Covid Problem\covid_vaccine_statewise.csv", header=True, inferSchema=True)

    # Create a temporary view
    covid_vaccine_statewisenew.createOrReplaceTempView("covid_vaccine_statewisenew")
    # Run the SQL query
    result = spark.sql("""
    SELECT
        `Updated On` as Date,
        State,
        ` Covaxin (Doses Administered)` * 100.0 / `Total Doses Administered` as Covaxin_Percentage,
        `CoviShield (Doses Administered)` * 100.0 / `Total Doses Administered` as CoviShield_Percentage,
        `Sputnik V (Doses Administered)` * 100.0 / `Total Doses Administered` as Sputnik_V_Percentage
    FROM 
        covid_vaccine_statewisenew
    ORDER BY Date ASC
""")

# Show the result
    result.show()
    spark.stop()

except Exception as e:
    print(e)


+----------+--------------------+-------------------+---------------------+--------------------+
|      Date|               State| Covaxin_Percentage|CoviShield_Percentage|Sputnik_V_Percentage|
+----------+--------------------+-------------------+---------------------+--------------------+
|01/02/2021|Dadra and Nagar H...|                0.0|                100.0|                NULL|
|01/02/2021|              Ladakh|                0.0|                100.0|                NULL|
|01/02/2021|Andaman and Nicob...|                0.0|                100.0|                NULL|
|01/02/2021|      Andhra Pradesh|  5.930021999072148|    94.06997800092785|                NULL|
|01/02/2021|   Arunachal Pradesh|                0.0|                100.0|                NULL|
|01/02/2021|               India|  2.070642173081831|    97.92935782691816|                NULL|
|01/02/2021|               Bihar| 2.3935931512996653|    97.60640684870033|                NULL|
|01/02/2021|          Chandiga

10. To find out percentage of people who took both the doses

In [25]:
try:
    spark = SparkSession.builder.appName("VaccinePercentage").getOrCreate()

    # Load data
    covid_vaccine_statewisenew = spark.read.csv(r"file:///C:\Users\sanju\OneDrive\Documents\dataset\Covid Problem\covid_vaccine_statewise.csv", header=True, inferSchema=True)

    # Create a temporary view
    covid_vaccine_statewisenew.createOrReplaceTempView("covid_vaccine_statewisenew")

    # Run the SQL query
    result = spark.sql("""
        SELECT
            covid_1.`Updated On` as Date,
            covid_1.State,
            COALESCE(covid_2.`Second Dose Administered`, 0) * 100.0 / COALESCE(covid_1.`First Dose Administered`, 1) as Percentage_Both_Doses
        FROM 
            covid_vaccine_statewisenew covid_1
        JOIN 
            covid_vaccine_statewisenew covid_2
        ON 
            covid_1.State = covid_2.State
            AND covid_1.`Updated On` = covid_2.`Updated On`
        ORDER BY 
            covid_1.`Updated On` ASC
    """)

    # Show the result
    result.show()
    spark.stop()

except Exception as e:
    print(e)


+----------+--------------------+---------------------+
|      Date|               State|Percentage_Both_Doses|
+----------+--------------------+---------------------+
|01/02/2021|Dadra and Nagar H...|                  0.0|
|01/02/2021|              Ladakh|                  0.0|
|01/02/2021|Andaman and Nicob...|                  0.0|
|01/02/2021|      Andhra Pradesh|                  0.0|
|01/02/2021|   Arunachal Pradesh|                  0.0|
|01/02/2021|               India|                  0.0|
|01/02/2021|               Bihar|                  0.0|
|01/02/2021|          Chandigarh|                  0.0|
|01/02/2021|        Chhattisgarh|                  0.0|
|01/02/2021|               Assam|                  0.0|
|01/02/2021|               Delhi|                  0.0|
|01/02/2021|                 Goa|                  0.0|
|01/02/2021|             Gujarat|                  0.0|
|01/02/2021|             Haryana|                  0.0|
|01/02/2021|    Himachal Pradesh|               